In [4]:
import os
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle
import numpy as np
from keras.models import Sequential
import gensim
from keras.layers.recurrent import LSTM,SimpleRNN
from sklearn.model_selection import train_test_split
from scipy import spatial
import numpy as np
from keras.models import load_model
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
import pandas as pd  
import re
nltk.download()
import sys
from bs4 import BeautifulSoup  
import csv
try:
    from nltk import wordpunct_tokenize
    from nltk.corpus import stopwords
except ImportError:
    print('[!] You need to install nltk (http://nltk.org/index.html)')

     

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [5]:
def _calculate_languages_ratios(text):
    languages_ratios = {}
    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios[language] = len(common_elements) # language "score"
    return languages_ratios

def detect_language(text):
    ratios = _calculate_languages_ratios(text)
    most_rated_language = max(ratios, key=ratios.get)
    return most_rated_language

def text_language(text):
    review_text = BeautifulSoup(text).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z0-9éàèùêâôî]", " ", review_text) 
    #3. detect language
    language = detect_language(letters_only)
    return language

def review_to_words(text):
    # 1. Remove HTML
    review_text = BeautifulSoup(text).get_text() 
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z0-9éàèùêâôî]", " ", review_text) 
    print(letters_only)
    #3. detect language
    language = detect_language(letters_only)
    print(language)
    stops = set(stopwords.words(language)) 
    # 4. Convert to lower case, split into individual words
    words = letters_only.lower().split()                                      
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    result = " ".join( meaningful_words ) 
    return result 



    


In [6]:
model = gensim.models.Word2Vec.load('300features_40minwords_10context')
file=open('conversation.json')
data = json.load(file)
cor=data["conversations"]

x=[]
y=[]


for i in range(len(cor)):
    for j in range(len(cor[i])):
        if j<len(cor[i])-1:
            x.append(cor[i][j])
            y.append(cor[i][j+1])

tok_x=[]
tok_y=[]
for i in range(len(x)):
    tok_x.append(nltk.word_tokenize(x[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))
    
    

sentend=np.ones((300),dtype=np.float32) 

vec_x=[]
for sent in tok_x:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_x.append(sentvec)
    
vec_y=[]
for sent in tok_y:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_y.append(sentvec)           
    
    
for tok_sent in vec_x:
    tok_sent[20:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_x:
    if len(tok_sent)<21:
        for i in range(21-len(tok_sent)):
            tok_sent.append(sentend)    
            
for tok_sent in vec_y:
    tok_sent[20:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_y:
    if len(tok_sent)<21:
        for i in range(21-len(tok_sent)):
            tok_sent.append(sentend)             
            
            
with open('conversation.pickle','wb') as f:
    pickle.dump([vec_x,vec_y],f) 

JSONDecodeError: Expecting value: line 229 column 2 (char 7705)

In [ ]:
with open('conversation.pickle', 'rb') as f:
    vec_x,vec_y=pickle.load(f)    
    
vec_x=np.array(vec_x,dtype=np.float64)
vec_y=np.array(vec_y,dtype=np.float64)    

x_train,x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)
    
model=Sequential()
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM500.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM1000.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM1500.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM2000.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM2500.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM3000.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM3500.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM4000.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM4500.h5');
model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
model.save('LSTM5000.h5');          
predictions=model.predict(x_test) 
mod = gensim.models.Word2Vec.load('300features_40minwords_10context');   
[mod.most_similar([predictions[10][i]])[0] for i in range(21)]

In [ ]:
model=load_model('LSTM5000.h5')
mod = gensim.models.Word2Vec.load('300features_40minwords_10context');
print('Hello friend, I am danny, a chatbot here for you to have fun and help you in your daily life')
print('I can speak with you in german, french, spanish or english')
print('My creator trained me on some stupid datasets as he was too lazy ti look for something better')
print('my knowledge of language comes from southpark I apologize if I hurt you :/')
print('I am specialised in Hamburg sorry if you live elsewhere')
while(True):
    x=input("Enter the message:");
    lng = text_language(x)
    if lng == 'german':
        sent_tokenize(x)
        blob = TextBlob(x)
        x = blob.translate(to='en')
        x = str(x)
        print('You: ',x)
    elif lng == 'french':
        sent_tokenize(x)
        blob = TextBlob(x)
        x = blob.translate(to='en')
        x = str(x)
        print('You: ',x)
    elif lng == 'spanish':
        sent_tokenize(x)
        blob = TextBlob(x)
        x = blob.translate(to='en')
        x = str(x)
        print('You: ',x)
    else:
        output_text = x
        print('You: ',x)
    sentend=np.ones((300),dtype=np.float32) 

    sent=nltk.word_tokenize(x.lower())
    sentvec = [mod[w] for w in sent if w in mod.wv.vocab]

    sentvec[20:]=[]
    sentvec.append(sentend)
    if len(sentvec)<21:
        for i in range(21-len(sentvec)):
            sentvec.append(sentend) 
    sentvec=np.array([sentvec])
    
    predictions = model.predict(sentvec)
    outputlist=[mod.most_similar([predictions[0][i]])[0][0] for i in range(21)]
    answer=' '.join(outputlist)
    
    answer = answer.replace("tripe", "")
    answer = answer.replace("labor", "")

    if lng == 'german':
        sent_tokenize(answer)
        blob = TextBlob(answer)
        answer = blob.translate(to='de')
        print('Danny the Bot: ',answer)
    elif lng == 'french':
        sent_tokenize(answer)
        blob = TextBlob(answer)
        answer = blob.translate(to='fr')
        print('Danny the Bot: ',answer)
    elif lng == 'spanish':
        sent_tokenize(answer)
        blob = TextBlob(answer)
        answer = blob.translate(to='es')
        print('Danny the Bot: ',answer)
    else:
        answer = answer
        print('Danny the Bot: ',answer)
